[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aderdouri/PINNs/blob/master/Tutorials/Burgers_1D_equation.ipynb)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define the neural network
class PINN(nn.Module):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layers) - 1):
            self.layers.append(nn.Linear(layers[i], layers[i+1]))

    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            x = torch.tanh(layer(x))
        x = self.layers[-1](x)
        return x

# Define the physics-informed loss
class PhysicsInformedLoss:
    def __init__(self, model, x_f, t_f, nu):
        self.model = model
        self.x_f = x_f
        self.t_f = t_f
        self.nu = nu

    def __call__(self):
        # Compute predictions
        X_f = torch.cat((self.x_f, self.t_f), dim=1).requires_grad_(True)
        u = self.model(X_f)

        # Compute partial derivatives
        u_t = torch.autograd.grad(u.sum(), self.t_f, create_graph=True)[0]
        u_x = torch.autograd.grad(u.sum(), self.x_f, create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x.sum(), self.x_f, create_graph=True)[0]

        # Burgers' equation residual
        f = u_t + u * u_x - self.nu * u_xx
        return torch.mean(f**2)

# Generate training data
def generate_training_data(N_u, N_f):
    # Boundary conditions
    x_left = torch.zeros((N_u, 1))
    x_right = torch.ones((N_u, 1))
    t_bc = torch.rand((N_u, 1))

    # Initial condition
    x_ic = torch.rand((N_u, 1))
    t_ic = torch.zeros((N_u, 1))

    u_left = torch.zeros((N_u, 1))
    u_right = torch.zeros((N_u, 1))
    u_ic = -torch.sin(np.pi * x_ic)

    # Collocation points
    x_f = torch.rand((N_f, 1))
    t_f = torch.rand((N_f, 1))

    # Boundary and initial data
    x_bc = torch.cat((x_left, x_right, x_ic), dim=0)
    t_bc = torch.cat((t_bc, t_bc, t_ic), dim=0)
    u_bc = torch.cat((u_left, u_right, u_ic), dim=0)

    return x_bc, t_bc, u_bc, x_f, t_f

In [ ]:
# Hyperparameters
layers = [2, 20, 20, 20, 1]
nu = 0.01 / np.pi
N_u = 50
N_f = 10000
epochs = 10000
lr = 1e-3

# Generate data
x_bc, t_bc, u_bc, x_f, t_f = generate_training_data(N_u, N_f)

# Initialize model and optimizer
model = PINN(layers)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(epochs):
    # Compute boundary loss
    X_bc = torch.cat((x_bc, t_bc), dim=1)
    u_pred = model(X_bc)
    loss_bc = torch.mean((u_pred - u_bc) ** 2)

    # Compute physics loss
    physics_loss_fn = PhysicsInformedLoss(model, x_f, t_f, nu)
    loss_f = physics_loss_fn()

    # Total loss
    loss = loss_bc + loss_f

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.5f}")

# Visualization
x = np.linspace(0, 1, 100)
t = np.linspace(0, 1, 100)
X, T = np.meshgrid(x, t)
X_star = torch.tensor(np.hstack((X.flatten()[:, None], T.flatten()[:, None])), dtype=torch.float32)

u_pred = model(X_star).detach().numpy()
U_pred = u_pred.reshape(X.shape)

plt.figure(figsize=(10, 5))
plt.imshow(U_pred, interpolation='nearest', cmap='rainbow', 
           extent=[0, 1, 0, 1], origin='lower', aspect='auto')
plt.colorbar(label='u(x,t)')
plt.xlabel('x')
plt.ylabel('t')
plt.title('Predicted Solution')
plt.show()
